In [29]:
import datasets 
data1 = datasets.load_dataset("derek-thomas/ScienceQA")

In [30]:
data2 = datasets.load_dataset("iamjoon/KorScienceQA")

In [31]:
data1, data2 

(DatasetDict({
     train: Dataset({
         features: ['image', 'question', 'choices', 'answer', 'hint', 'task', 'grade', 'subject', 'topic', 'category', 'skill', 'lecture', 'solution'],
         num_rows: 12726
     })
     validation: Dataset({
         features: ['image', 'question', 'choices', 'answer', 'hint', 'task', 'grade', 'subject', 'topic', 'category', 'skill', 'lecture', 'solution'],
         num_rows: 4241
     })
     test: Dataset({
         features: ['image', 'question', 'choices', 'answer', 'hint', 'task', 'grade', 'subject', 'topic', 'category', 'skill', 'lecture', 'solution'],
         num_rows: 4241
     })
 }),
 DatasetDict({
     train: Dataset({
         features: ['image', 'question', 'choices', 'answer', 'hint', 'task', 'grade', 'subject', 'topic', 'category', 'skill', 'lecture', 'solution', 'korean_question', 'korean_choices', 'korean_hint'],
         num_rows: 12726
     })
 }))

In [32]:
korean_columns = ['korean_question', 'korean_choices', 'korean_hint']
korean_data = data2['train'].select_columns(korean_columns)

# data1에 한국어 컬럼 추가
new_data1 = data1['train']
for column in korean_columns:
    new_data1 = new_data1.add_column(column, korean_data[column])

new_data1

Dataset({
    features: ['image', 'question', 'choices', 'answer', 'hint', 'task', 'grade', 'subject', 'topic', 'category', 'skill', 'lecture', 'solution', 'korean_question', 'korean_choices', 'korean_hint'],
    num_rows: 12726
})

In [33]:
new_data1["korean_choices"][0]

"['웨스트버지니아', '루이지애나', '애리조나', '오클라호마']"

In [34]:
import ast 
def string_to_list(string):
    return ast.literal_eval(string)

# 데이터셋의 korean_choices 컬럼 변환
new_korean_choices = [string_to_list(choice) for choice in new_data1['korean_choices']]
new_data1 = new_data1.remove_columns(['korean_choices'])
new_data1 = new_data1.add_column('korean_choices', new_korean_choices)
new_data1["korean_choices"][0]


['웨스트버지니아', '루이지애나', '애리조나', '오클라호마']

In [35]:
data = new_data1

In [36]:
data["choices"][0]

['West Virginia', 'Louisiana', 'Arizona', 'Oklahoma']

In [37]:
data["answer"][0]

0

In [38]:
data["image"][:6]

[<PIL.PngImagePlugin.PngImageFile image mode=RGB size=750x429>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=302x232>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=302x232>,
 None,
 None,
 None]

In [39]:
from PIL import Image
import os

def save_image_and_get_path(image_obj, image_id, save_dir):
    image_path = os.path.join(save_dir, f'image_{image_id}.jpeg')
    image_obj.save(image_path)
    return image_path

# 이미지 데이터를 저장할 경로
save_dir = "./image"
os.makedirs(save_dir, exist_ok=True)

# 데이터셋에서 이미지 객체 추출 및 저장
for idx, example in enumerate(data):
    if idx > 200:  
        break 
    if isinstance(example['image'], Image.Image):  # 이미지가 PIL.Image일 경우
        image_path = save_image_and_get_path(example['image'], idx, save_dir)
        example['image'] = image_path  # 경로를 저장

In [40]:
# answer 인덱스를 choices에서 찾아와 str로 변환
def convert_answer_to_choice(example):
    # answer는 choices의 인덱스에 해당하므로 이를 문자열로 변환
    example['answer_str'] = example['korean_choices'][example['answer']]
    return example

# map 함수를 이용해 각 데이터에 대해 answer를 변환
data = data.map(convert_answer_to_choice)

# 변환 결과 확인
print(data[0]['answer_str'])


웨스트버지니아


In [41]:
len(image_path)

22

In [42]:

from glob import glob 
from natsort import natsorted
image_path = natsorted(glob("./image/*.jpeg"))
image_path[0:10]

['./image/image_0.jpeg',
 './image/image_1.jpeg',
 './image/image_2.jpeg',
 './image/image_6.jpeg',
 './image/image_7.jpeg',
 './image/image_9.jpeg',
 './image/image_10.jpeg',
 './image/image_11.jpeg',
 './image/image_13.jpeg',
 './image/image_16.jpeg']

In [43]:
import datasets
from PIL import Image
import os
from glob import glob

# 주어진 이미지 경로 로드
image_dir = "./image"
image_files = sorted(glob(os.path.join(image_dir, "*.jpeg")))

# 파일명에서 인덱스만 추출하는 함수
def get_image_index(image_path):
    filename = os.path.basename(image_path)
    return int(filename.split('_')[1].split('.')[0])

# 인덱스와 이미지 경로를 매핑
image_map = {get_image_index(image_path): image_path for image_path in image_files}

# 주어진 데이터셋 로드
dataset = data

# conversation 포맷으로 변환하는 함수 정의
def convert_to_conversation(example, idx):
    # 이미지가 있는지 확인하여 없으면 질문만 포함
    if idx in image_map:
        conversation = [
            {
                "from": "user",
                "value": f"<img>{image_map[idx]}</img>\n{example['korean_question']}"
            },
            {
                "from": "user",
                "value": f"Choices: {example['korean_choices']}"
            },
            {
                "from": "assistant",
                "value": f"Answer: {example['answer_str']}"
            },
            {
                "from": "assistant",
                "value": f"Hint: {example['korean_hint']}"
            }
        ]
    else:
        # 이미지가 없는 경우
        conversation = [
            {
                "from": "user",
                "value": f"{example['korean_question']}"
            },
            {
                "from": "user",
                "value": f"Choices: {example['korean_choices']}"
            },
            {
                "from": "assistant",
                "value": f"Answer: {example['answer_str']}"
            },
            {
                "from": "assistant",
                "value": f"Hint: {example['korean_hint']}"
            }
        ]
    
    return {"conversations": conversation}

# 변환된 데이터셋을 위해 map 함수를 사용하여 각 split에 대해 처리
converted_train = dataset.select(range(200)).map(lambda example, idx: convert_to_conversation(example, idx), with_indices=True)
# converted_validation = dataset['validation'].select(range(100)).map(lambda example, idx: convert_to_conversation(example, idx), with_indices=True)
# converted_test = dataset['test'].select(range(100)).map(lambda example, idx: convert_to_conversation(example, idx), with_indices=True)

# 변환된 데이터셋을 새로운 DatasetDict로 묶기
converted_dataset = datasets.DatasetDict({
    "train": converted_train,
    # "validation": converted_validation,
    # "test": converted_test
})

# 변환된 데이터셋 출력 확인
print(converted_dataset)

# 원한다면 변환된 데이터셋을 디스크에 저장 가능
# converted_dataset.save_to_disk('path_to_save_converted_dataset')


Map: 100%|██████████| 200/200 [00:00<00:00, 3998.63 examples/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'question', 'choices', 'answer', 'hint', 'task', 'grade', 'subject', 'topic', 'category', 'skill', 'lecture', 'solution', 'korean_question', 'korean_hint', 'korean_choices', 'answer_str', 'conversations'],
        num_rows: 200
    })
})


In [47]:
converted_dataset["train"]["conversations"][0]

[{'from': 'user',
  'value': '<img>./image/image_0.jpeg</img>\n다음 주 중에서 가장 북쪽에 있는 곳은 어디인가요?'},
 {'from': 'user', 'value': "Choices: ['웨스트버지니아', '루이지애나', '애리조나', '오클라호마']"},
 {'from': 'assistant', 'value': 'Answer: 웨스트버지니아'},
 {'from': 'assistant', 'value': 'Hint: '}]

# Qwen-vl-chat 포맷으로 변경

In [45]:
import json

# 데이터셋에서 conversations 부분만 추출 (converted_dataset는 이미 정의되어 있어야 합니다)
conversations_list = converted_dataset["train"]["conversations"]

# 새로운 포맷에 맞춰 데이터를 변환
formatted_conversations = []

for idx, conversation in enumerate(conversations_list):
    conversation_entry = {
        "id": f"identity_{idx}",  # 각 대화에 고유 ID 부여
        "conversations": conversation  # 기존 conversations 추가
    }
    formatted_conversations.append(conversation_entry)


In [46]:

# 변환된 데이터를 JSON 파일로 저장
with open('20241016_formatted_conversations.json', 'w') as f:
    json.dump(formatted_conversations, f, indent=4, ascii=False)

print("데이터가 formatted_conversations.json 파일로 저장되었습니다.")


데이터가 formatted_conversations.json 파일로 저장되었습니다.


In [1]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    "/home/datadriven/ds.kang/Multi-modal-Finetuning/output_qwen", # path to the output directory
    device_map="auto",
    trust_remote_code=True
).eval()

/home/datadriven/anaconda3/envs/qwen_tuning/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/datadriven/anaconda3/envs/qwen_tuning/lib/python3.9/site-packages/transformers/utils/generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/home/datadriven/anaconda3/envs/qwen_tuning/lib/python3.9/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, pl

In [51]:
conversations_list[103][:2]

[{'from': 'user',
  'value': '<img>./image/image_103.jpeg</img>\n어떤 용액이 녹색 입자의 농도가 더 높나요?'},
 {'from': 'user', 'value': "Choices: ['용액 A', '둘 다 아님; 농도가 동일함', '용액 B']"}]

In [25]:
from transformers import AutoTokenizer 
tokenizer = AutoTokenizer.from_pretrained("/home/datadriven/ds.kang/Multi-modal-Finetuning/output_qwen", trust_remote_code=True)

/home/datadriven/anaconda3/envs/qwen_tuning/lib/python3.9/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [52]:
image_path = 'https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg'
response, history = model.chat(tokenizer, query=f'{conversations_list[101][:2]}', history=None)
print(response)

공통점은 화학적 변화입니다.


In [53]:
conversations_list[101][2]

{'from': 'assistant', 'value': 'Answer: 둘 다 화학적 변화입니다.'}